In [ ]:
import numpy as np
import numpy.random as rd

class BanditEXP4:
    def __init__(self, n_experts, k_arms, loss, eta=None):
        self.N = n_experts  # Nombre d'experts
        self.K = k_arms     # Nombre de bras
        self.eta = eta if eta is not None else 0.1  # Learning rate
        
        # Distribution de confiance sur les EXPERTS (taille N, pas K)
        self.q = np.ones(self.N) / self.N 
        
        # Poids (non normalisés) pour stabilité numérique
        self.weights = np.ones(self.N)

        self.loss = loss

    def get_arm_distribution(self, expert_advice):
        """
        Calcule p_t (distrib sur les bras) à partir de q_t (confiance experts)
        et expert_advice (xi).
        
        expert_advice: Matrice (N, K) où [i, k] est la probabilité que l'expert i recommande le bras k.
        """
        # p[k] = somme(q[i] * xi[i, k]) sur tous les experts i
        # C'est un produit matriciel : q (1xN) dot xi (NxK) -> (1xK)
        p = np.dot(self.q, expert_advice)
        
        p /= np.sum(p) # Renormalisation
        return p

    def sample_arm(self, p):
        """Tire un bras selon la distribution p"""
        return rd.choice(np.arange(self.K), p=p)

    def update(self, chosen_arm, observed_loss, expert_advice):
        """
        Mise à jour des poids selon EXP4.
        
        chosen_arm: index du bras joué (It)
        observed_loss: perte réelle subie (l_t)
        expert_advice: Matrice (N, K) des conseils à ce tour
        """
        # 1. Recalculer p pour s'assurer qu'on a le p utilisé pour le tirage
        p = self.get_arm_distribution(expert_advice)
        p_chosen = p[chosen_arm]

        # 2. Estimer la perte pour TOUS les bras (Estimateur sans biais)
        # Lt_hat[k] = loss / p[k] si k == chosen_arm sinon 0
        Lt_hat = np.zeros(self.K)
        Lt_hat[chosen_arm] = observed_loss / p_chosen

        # 3. Estimer la perte pour chaque EXPERT
        # y_hat[i] = produit scalaire(conseil de l'expert i, perte estimée des bras)
        # y_hat shape: (N,)
        y_hat = np.dot(expert_advice, Lt_hat)

        # 4. Mise à jour des poids des experts (Exponential Weighting)
        # w_{t+1} = w_t * exp(-eta * y_hat)
        self.weights = self.weights * np.exp(-self.eta * y_hat)
        
        # 5. Normalisation pour obtenir q
        self.q = self.weights / np.sum(self.weights)
        
        return self.q

def run_simulation():
    N_EXPERTS = 3
    K_ARMS = 2
    T_STEPS = 100
    
    # Instanciation
    # Eta théorique pour T=100
    eta = np.sqrt(2 * np.log(N_EXPERTS) / (K_ARMS * T_STEPS))
    exp4 = BanditEXP4(n_experts=N_EXPERTS, k_arms=K_ARMS, eta=eta)
    
    total_loss = 0
    
    print(f"Début simulation: {N_EXPERTS} experts, {K_ARMS} bras")

    for t in range(T_STEPS):
        # 1. Générer des conseils d'experts (fictifs pour l'exemple)
        # Chaque expert donne une proba sur les bras. Somme sur l'axe 1 doit faire 1.
        xi = rd.dirichlet(np.ones(K_ARMS), size=N_EXPERTS)
        
        # 2. EXP4 décide des probas de bras
        p_arms = exp4.get_arm_distribution(xi)
        
        # 3. On tire un bras
        chosen_arm = exp4.sample_arm(p_arms)
        
        # 4. Environnement renvoie une perte (Ex: bras 0 gagne souvent, bras 1 perd)
        # Disons que le bras 0 a une perte moyenne de 0.2 et le bras 1 de 0.8
        real_loss = rd.binomial(1, 0.2 if chosen_arm == 0 else 0.8)
        
        # 5. Mise à jour
        q_new = exp4.update(chosen_arm, real_loss, xi)
        
        total_loss += real_loss

    print(f"Fin. Perte cumulée: {total_loss}")
    print(f"Confiance finale envers les experts: {np.round(exp4.q, 3)}")

run_simulation()

Début simulation: 3 experts, 2 bras
Fin. Perte cumulée: 49
Confiance finale envers les experts: [0.317 0.47  0.214]
